In [ ]:
# Importamos bibliotecas.
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Definimos el sistema diferencial.
def dm(r, w, m, s, F, G, h):
    N = 1 - 2*m/r
    return (r**2/2)*( ((s*N*G)/w)**2 + G**2*N + (F**2)/(N*s**2) )
                 
def ds(r, w, m, s, F, G, h):
    N = 1 - 2*m/r
    return r*s*( G**2 + (F/(N*s))**2 )

def dF(r, w, m, s, F, G, h):
    N = 1 - 2*m/r
    return (w-(s**2*N)/w)*G

def dG(r, w, m, s, F, G, h):
    N = 1 - 2*m/r
    Ds = r*s*( G**2 + (F/(N*s))**2 )
    DN = 2*(m/r**2)-(2/r)*( (r**2/2)*( ((s*N*G)/w)**2 + G**2*N + (F**2)/(N*s**2) ) )
    return - ( Ds/s + 2/r + DN/N )*G - (w/(s**2*N**2))*F

In [ ]:
# Definimos el RK4 en sí.
def rk4(r, w, m, s, F, G, h):
    k1m = dm(r, w, m, s, F, G, h)
    k1s = ds(r, w, m, s, F, G, h)
    k1F = dF(r, w, m, s, F, G, h)
    k1G = dG(r, w, m, s, F, G, h)
    
    k2m = dm(r + 0.5*h, w, m + 0.5*k1m*h, s + 0.5*k1s*h, F + 0.5*k1F*h, G + 0.5*k1G*h, h)
    k2s = ds(r + 0.5*h, w, m + 0.5*k1m*h, s + 0.5*k1s*h, F + 0.5*k1F*h, G + 0.5*k1G*h, h)
    k2F = dF(r + 0.5*h, w, m + 0.5*k1m*h, s + 0.5*k1s*h, F + 0.5*k1F*h, G + 0.5*k1G*h, h)
    k2G = dG(r + 0.5*h, w, m + 0.5*k1m*h, s + 0.5*k1s*h, F + 0.5*k1F*h, G + 0.5*k1G*h, h) 
    
    k3m = dm(r + 0.5*h, w, m + 0.5*k2m*h, s + 0.5*k2s*h, F + 0.5*k2F*h, G + 0.5*k2G*h, h)
    k3s = ds(r + 0.5*h, w, m + 0.5*k2m*h, s + 0.5*k2s*h, F + 0.5*k2F*h, G + 0.5*k2G*h, h)
    k3F = dF(r + 0.5*h, w, m + 0.5*k2m*h, s + 0.5*k2s*h, F + 0.5*k2F*h, G + 0.5*k2G*h, h)
    k3G = dG(r + 0.5*h, w, m + 0.5*k2m*h, s + 0.5*k2s*h, F + 0.5*k2F*h, G + 0.5*k2G*h, h) 
    
    k4m = dm(r + h, w, m + k3m*h, s + k3s*h, F + k3F*h, G + k3G*h, h)
    k4s = ds(r + h, w, m + k3m*h, s + k3s*h, F + k3F*h, G + k3G*h, h)
    k4F = dF(r + h, w, m + k3m*h, s + k3s*h, F + k3F*h, G + k3G*h, h)
    k4G = dG(r + h, w, m + k3m*h, s + k3s*h, F + k3F*h, G + k3G*h, h) 
    
    mrk4 = m + (k1m + 2*k2m + 2*k3m + k4m)*(h/6)
    srk4 = s + (k1s + 2*k2s + 2*k3s + k4s)*(h/6)
    Frk4 = F + (k1F + 2*k2F + 2*k3F + k4F)*(h/6)
    Grk4 = G + (k1G + 2*k2G + 2*k3G + k4G)*(h/6)
    
    return [mrk4, srk4, Frk4, Grk4]

In [ ]:
# Definimos los parámetros de entrada.
F1, w, rf = 0.19, 1.2594938327, 46

In [ ]:
# Definimos las condiciones iniciales.
h  = 0.01 # Tamaño del paso.
r0 = h # posición inicial.

m0 = (F1**2/6)*r0**3 # Condición inicial para m(r).
s0 = 1 + (F1**2/2)*r0**2 # Condición inicial para \sigma(r).
F0 = F1 + ( (F1*(1-w**2))/6 )*r0**2 # Condición inicial para F(r).
G0 = -F1*w*r0/3 # Condición inicial para G(r).

In [ ]:
# Comenzamos a solucionar.

# Definimos los arreglos donde guardaremos la solución.
r = [ ]
m = [ ]
s = [ ]
F = [ ]
G = [ ]

# Metemos las condiciones iniciales.
r.append(r0)
m.append(m0)
s.append(s0)
F.append(F0)
G.append(G0)

# Definimos un contador de índices.
c = 0

# Llenamos los arreglos.
for i in np.arange(r0+h, rf+h, h):
    r.append(i)
    m.append( rk4(r[c], w, m[c], s[c], F[c], G[c], h)[0] )
    s.append( rk4(r[c], w, m[c], s[c], F[c], G[c], h)[1] )
    F.append( rk4(r[c], w, m[c], s[c], F[c], G[c], h)[2] )
    G.append( rk4(r[c], w, m[c], s[c], F[c], G[c], h)[3] )
    c += 1

In [ ]:
# Graficamos.
fig = plt.figure(figsize=(15, 10))
fig.tight_layout()

# r vs m.
ax = plt.subplot(2,2,1)
ax.plot(r, m, color = 'maroon')
ax.set_xlabel(r'$r$')
ax.set_ylabel(r'$m$')
ax.grid(True)
ax.set_title(r'Gŕafica de $r$ vs $m$')

# r vs s.
ax = plt.subplot(2, 2, 2)
ax.plot(r, s, color = 'orangered')
ax.set_xlabel(r'$r$')
ax.set_ylabel(r'$\sigma$')
ax.grid(True)
ax.set_title(r'Gŕafica de $r$ vs $\sigma$')

# r vs f.
ax = plt.subplot(2, 2, 3)
ax.plot(r, F, color = 'blue')
ax.set_xlabel(r'$r$')
ax.set_ylabel(r'$F$')
ax.grid(True)
ax.set_title(r'Gŕafica de $r$ vs $F$')

# r vs g.
ax = plt.subplot(2,2, 4)
ax.plot(r, G, color = 'red')
ax.set_xlabel(r'$r$')
ax.set_ylabel(r'$G$')
ax.grid(True)
ax.set_title(r'Gŕafica de $r$ vs $G$')

plt.show()

In [ ]:
# Regresamos una adimensión.
s00 = 1/s[-1]
wt = w*s00

# Imprimimos parámetros importantes.
print(f'f1  = {F1}')
print(f'w   = {w}')
print(f'rf  = {rf}')
print(f'wt  = {wt}')
print(f'mass = {m[-1]}')

# Calculamos m95, r95 y la compacidad.   
m95 = m[-1]*0.95

for i in range(1,len(m)):
    if m[-i] <= m95:
        break

# Imprimimos esta información.
print(f'm95 = {m[-i]}')
print(f'r95 = {r[-i]}')
print(f'com = {m[-i]/r[-i]}')

# Calculamos la corriente conservada.
r = np.array(r)
s = np.array(s)
G = np.array(G)
m = np.array(m)
N = np.ones(len(r)) - 2*m/r
j = (1/w)*(r**2*G**2*s*N)

# Calculamos la carga de Nöther.
Q = 0

for i in range(len(r)-2):
    area = (j[i]+4*j[i+1]+j[i+2])*h/6
    Q += area

# Imprimimos la carga.
print(f'Q   = {Q}')